# Data Preprocessing 

## Schritte:

1. Einlesen der Reviews
2. Preprocessing Schritte durchführen
3. Ausgeben der vereinfachten Liste 



## 1. Einlesen der Reviews


Importiere der benötigten Bibliotheken.
Installation der benötigten Module.
Konsoleneingabe:

pip install contractions
pip install inflect

In [1]:
import re, string, unicodedata
import nltk
import contractions
import inflect
import pandas as pd
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [2]:
nltk.download('punkt') # necessary package for word_tokenize
nltk.download('stopwords') # necessary package for remove_stopwords
nltk.download('wordnet') # necessary package for lemmatize_verbs

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Importieren der Reviews in ein Datafile (df).

In [3]:
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)
df = list(parse('../Data_raw/reviews_Musical_Instruments_5.json.gz'))

Testausgabe des ersten Reviewtext aus dem Dataframe.

In [4]:
df[0]["reviewText"]

"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,"

## 2. Data Preprocessing Schritte durchführen

### 2.1 Contractions

Wandelt englische Wörter wie: "you've, it's, don't" zu deren ausgeschriebener Grundform um.

Eingabe: Belieber Text als String:
"It's a nice day."

Ausgabe: Text als String:
"It is a nice day"


In [5]:
def replace_contractions(text):
    return contractions.fix(text)

In [6]:
text = "Hello, it's a very nice day."
print(replace_contractions(text))

Hello, it is a very nice day.


### 2.2 Tokenize

Wandelt alle Wörter und Satzzeichen eines Textes in eine Liste mit einzelenen Wörtern um.

Eingabe: Beliebiger Text als String: 
"It's a nice day."

Ausgabe: Liste mit Wörtner und Satzzeichen als Listenelemente: 
['Hello', ',', 'it', "'s", 'a', 'very', 'nice', 'day', '.']

In [7]:
def tokenize_words(text):
    return nltk.word_tokenize(text)

In [8]:
text2 = "Hello, it's a very nice day."
print(tokenize_words(text2))

['Hello', ',', 'it', "'s", 'a', 'very', 'nice', 'day', '.']


### 2.3 Non-ascii entfernen 

Entfernt alle nicht Ascii zeichen aus einer Liste mit tokenized Wörten.

Eingabe: Liste mit tokenized Words: ['Hello', ',', 'it', "'s", 'a', 'very', 'nice', 'day', '.']

Ausgabe: Liste nur mit Ascii Zeichen: ['Hello', ',', 'it', "'s", 'a', 'very', 'nice', 'day', '.']
    

In [9]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [10]:
liste1 = ['Hello', '我', 'it', "'s", '我', 'very', 'nice', 'day', '.']
print(remove_non_ascii(liste1))

['Hello', '', 'it', "'s", '', 'very', 'nice', 'day', '.']


### 2.4 Convert to Lowercase

Konvertiert alle Zeichen der Liste in Kleinbuchstaben

Einagbe: Liste mit Klein-und Großbuchstaben: ['Hello', 'it', 'is', 'a', 'nice', 'day' ]

Ausgabe: Liste mit Kleinbuchstaben: ['hello', 'it', 'is', 'a', 'nice', 'day' ]

In [11]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [12]:
liste2 = ['Hello', 'it', 'is', 'a', 'nice', 'day' ]
print (to_lowercase(liste2))

['hello', 'it', 'is', 'a', 'nice', 'day']


## 2.5 Remove Punctuation

Entfernt alle Satzzeichen aus der Liste 

Eingabe: Liste mit Satzzeichen: ['Hello','?',it', 'is', 'a', 'nice', 'day','.']

Ausgabe: Liste ohne Satzzeichen: ['Hello','it', 'is', 'a', 'nice', 'day',]

In [13]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [14]:
liste3 = ['Hello','?','it', 'is', 'a', 'nice', 'day','.']
print(remove_punctuation(liste3))

['Hello', 'it', 'is', 'a', 'nice', 'day']


## 2.6 Replace Numbers

Ersetzt alle Intenger der Liste durch eine textuelle Darstellung

Einagbe: Liste mit Intenger: ['Hello','it', 'is', 'a', 'nice', 'day', '10']

Ausgabe: Liste mit ausgeschrieben Inetnger: ['Hello','it', 'is', 'a', 'nice', 'day', 'ten']

In [15]:
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

In [16]:
liste4 = ['Hello','it', 'is', 'a', 'nice', 'day', '10']
print(replace_numbers(liste4))

['Hello', 'it', 'is', 'a', 'nice', 'day', 'ten']


## 2.7 Remove Stopwords

Entfertn alle Stoppwörter aus der Liste (Wörter müssen kleinkeschrieben sein. Zuerst "Convert to Lowercase" anwenden)

Eingabe: Liste mit Stoppwörter: ['I','like ', 'reading', 'so', 'I', 'red']

Ausgabe: Liste ohne Stoppwörter:['like', 'reading', 'red']

In [17]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

In [18]:
liste5 = ['I','like ', 'reading', 'so', 'I', 'red', 'you']
print(remove_stopwords(liste5))

['I', 'like ', 'reading', 'I', 'red']


## 2.8 Stem Words

Reduzieren von  abgeleiteten Wörtern auf ihren Stamm, ihre Basis oder ihre Wurzelform - im Allgemeinen eine schriftliche Wortform. Konvertiert alle Zeichen der Liste in Kleinbuchstaben

Eingabe: ["reading", "booking"]

Ausgabe: ["read", "book"]

In [19]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems


In [20]:
liste6 = ['I','am', 'reading', 'so', 'done', 'I', 'booking']
print(stem_words(liste6))

['i', 'am', 'read', 'so', 'don', 'i', 'book']


## 2.9 Lemmatize Verbs

Gruppierung der verschiedenen gebeugten Formen eines Wortes

Wandelt in Präsens um und reduziert Wörtern auf ihren Stamm

Eingabe: ["am", "done"]

Ausgabe: ["be", "do"]

In [21]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [22]:
liste7 = ['I','am', 'reading', 'done', 'I', 'booking']
print(lemmatize_verbs(liste7))

['I', 'be', 'read', 'do', 'I', 'book']


## 3.0 Normalize

Funktion für die Anwendung aller Data Preprocessing Schritten

In [23]:
def preprocessing_steps(words):
    words = replace_contractions(words) # wandelt "you've, it's, don't" zu deren ausgeschriebener Grundform um
    words = tokenize_words(words) # wandelt alle Wörter und Satzzeichen eines Textes in eine Liste um
    words = remove_non_ascii(words) # entfernt alle nicht Ascii Zeichen aus einer Liste mit tokenized Wörten (leere Strings bleiben)
    words = to_lowercase(words) # konvertiert alle Zeichen der Liste in Kleinbuchstaben
    words = remove_punctuation(words) # entfernt alle Satzzeichen aus der Liste
    words = replace_numbers(words) # ersetzt alle Intenger der Liste durch eine textuelle Darstellung
    words = remove_stopwords(words) # entfertn alle Stoppwörter aus der Liste (Wörter müssen kleinkeschrieben sein. Zuerst "Convert to Lowercase" anwenden)
    # words = stem_words(words) # reduziert Wörtern auf ihren Stamm und konvertiert alle Zeichen der Liste in Kleinbuchstaben
    # words = lemmatize_verbs(words) # wandelt in Präsens um und reduziert Wörtern auf ihren Stamm
    return words


Ordner für gefilterte Reviewkategorien (.csv) anlegen

In [24]:
import os
if os.path.exists("../Data_filtered"):
    print("Ordner Data_filtered existiert bereits")
if not os.path.exists("../Data_filtered"):
    os.makedirs("../Data_filtered")
    print("Ordner Data_filtered wurde erfolgreich angelegt")

Ordner Data_filtered existiert bereits


Funktion normalisiert eine Reviewkategorie und erstellt eine .csv Datei und speichert diese in dem Ordner Data_filtered ab

In [25]:
import time
def read_preprocess_csv(path, name):
    if os.path.isfile("../Data_filtered/%s.csv" %name):
        return print("Die gefilterte CSV existiert bereits")
    else:
        start_time = time.time()
        print("Die gefilterte CSV wird angelegt")
        data_raw = list(parse(path))
        reviewText = []
        reviewRating = []
        for i in range(len(data_raw)): 
            text = preprocessing_steps(data_raw[i]["reviewText"])
            score = data_raw[i]["overall"]
            reviewText.append(text)
            reviewRating.append(score)
        data_filtered = pd.DataFrame(data={"review": reviewText, "rating": reviewRating})
        data_filtered.to_csv("../Data_filtered/%s.csv" %name, sep=';',index=False)
        end_time = time.time()
        return print("Datei erfolgreich angelegt \n" 'Dauer: {:5.3f}min'.format((end_time-start_time)/60))

In [26]:
read_preprocess_csv("../Data_raw/reviews_Musical_Instruments_5.json.gz", "reviews_Musical_Instruments")

Die gefilterte CSV wird angelegt
Datei erfolgreich angelegt 
Dauer: 22.623min


In [45]:
read_preprocess_csv("../Data_raw/reviews_Automotive_5.json.gz", "reviews_Automotive")

Die gefilterte CSV existiert bereits
